# Inspect Mask R-CNN Model with COCO Precision

In [46]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = './'


In [47]:
from pycocotools.coco import COCO
import mrcnn.model as modellib
import os

# Define the paths to your dataset and model
DATA_DIR = os.path.join(ROOT_DIR, 'dataset')
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_VAL_ANNOTATIONS = os.path.join(DATA_DIR, 'val', 'annotations.json')

# Initialize COCO validation dataset
coco = COCO(COCO_VAL_ANNOTATIONS)
image_ids = coco.getImgIds()

# Load validation dataset
dataset_val = cElegans.CElegansDataset()
dataset_val.load_c_elegans(DATA_DIR, "val")  # Load validation data
dataset_val.prepare()

# Load the pre-trained Mask R-CNN model in inference mode
class InferenceConfig(cElegans.CElegansConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5 


config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=MODEL_DIR)

# Load the weights from the trained model
model.load_weights('mask_rcnn_custom.h5', by_name=True)


loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


2024-09-10 16:24:08.639344: W tensorflow/c/c_api.cc:304] Operation '{name:'res4b_branch2c_7/bias/Assign' id:70036 op device:{requested: '', assigned: ''} def:{{{node res4b_branch2c_7/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](res4b_branch2c_7/bias, res4b_branch2c_7/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [34]:
# Function to convert Mask R-CNN detection results to COCO format
def format_coco_result(image_id, r):
    results = []
    for i in range(len(r['class_ids'])):
        y1, x1, y2, x2 = r['rois'][i]
        w = x2 - x1
        h = y2 - y1
        # Ensure the width and height are positive
        if w > 0 and h > 0:
            result = {
                "image_id": image_id,
                "category_id": int(r['class_ids'][i]),
                "bbox": [float(x1), float(y1), float(w), float(h)],
                "score": float(r['scores'][i])
            }
            results.append(result)
    return results



In [35]:
import json

# Perform inference on each image and save results in COCO format
results = []
for image_id in image_ids:
    image = dataset_val.load_image(image_id)
    r = model.detect([image], verbose=1)[0]
    coco_results = format_coco_result(image_id, r)
    results.extend(coco_results)

# Save the detection results to a JSON file
with open('detection_results.json', 'w') as f:
    json.dump(results, f)

Processing 1 images
image                    shape: (1012, 1012, 3)       min:    4.00000  max:  161.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   57.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int64
anchors                  shape: (1, 261888, 4)        min:   -0.17695  max:    1.11439  float32


/opt/anaconda3/envs/cv_env/lib/python3.9/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-09-10 15:01:31.899886: W tensorflow/c/c_api.cc:304] Operation '{name:'bn4t_branch2c_4/beta/Assign' id:44718 op device:{requested: '', assigned: ''} def:{{{node bn4t_branch2c_4/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](bn4t_branch2c_4/beta, bn4t_branch2c_4/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-09-10 15:01:37.330492: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "CropAndResize"

Processing 1 images
image                    shape: (1012, 1012, 3)       min:   46.00000  max:  158.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   54.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int64
anchors                  shape: (1, 261888, 4)        min:   -0.17695  max:    1.11439  float32
Processing 1 images
image                    shape: (1012, 1012, 3)       min:    3.00000  max:  176.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   72.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int64
anchors                  shape: (1, 261888, 4)        min:   -0.17695  max:    1.11439  float32
Processing 1 images
image                    shape: (1012, 1012, 3)       min:    3.00000  max:  159.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   55.10000

KeyboardInterrupt: 

In [51]:
from pycocotools.cocoeval import COCOeval

cocoGt = COCO(COCO_VAL_ANNOTATIONS)  
cocoDt = cocoGt.loadRes('detection_results.json') 

coco_eval = COCOeval(cocoGt, cocoDt, 'bbox')  # Use 'segm' if evaluating masks
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


loading annotations into memory...
Done (t=0.11s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.08s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.00s).
Accumulating evaluation results...
DONE (t=0.42s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe